# AdventureWorks Sales Data Cleaning Script
---

This script performs a full cleaning process on the AdventureWorks Sales dataset.

### Key Tasks:
- Loads all sheets from the Excel source file
- Detects and reports missing or placeholder values
- Cleans placeholder customer and reseller records (`-1`)
- Ensures referential integrity across related tables (`Sales_data`)
- Converts date keys from integer to standard date format
- Verifies uniqueness of key columns
- Saves the cleaned dataset to a new Excel file

This script is intended as a preprocessing step before building dashboards or running deeper analysis.

---
---
---

### 🔍 Inspect Sheet Structure

This section loads the Excel file and prints the column headers for each sheet. It's useful for quickly understanding the structure and schema of the dataset before performing any data cleaning or analysis.

---

In [10]:
import pandas as pd

# Load the Excel file
file_path = r"data\AdventureWorks Sales.xlsx"
xls = pd.ExcelFile(file_path)

# Print headers for each sheet
print("Column Headers in Each Sheet:\n")
for sheet in xls.sheet_names:
    df = xls.parse(sheet, nrows=0)  # Load only header
    print(f"{sheet}:")
    print(list(df.columns))
    print("-" * 50)


Column Headers in Each Sheet:

Sales Order_data:
['Channel', 'SalesOrderLineKey', 'Sales Order', 'Sales Order Line']
--------------------------------------------------
Sales Territory_data:
['SalesTerritoryKey', 'Region', 'Country', 'Group']
--------------------------------------------------
Sales_data:
['SalesOrderLineKey', 'ResellerKey', 'CustomerKey', 'ProductKey', 'OrderDateKey', 'DueDateKey', 'ShipDateKey', 'SalesTerritoryKey', 'Order Quantity', 'Unit Price', 'Extended Amount', 'Unit Price Discount Pct', 'Product Standard Cost', 'Total Product Cost', 'Sales Amount']
--------------------------------------------------
Reseller_data:
['ResellerKey', 'Reseller ID', 'Business Type', 'Reseller', 'City', 'State-Province', 'Country-Region', 'Postal Code']
--------------------------------------------------
Date_data:
['DateKey', 'Date', 'Fiscal Year', 'Fiscal Quarter', 'Month', 'Full Date', 'MonthKey']
--------------------------------------------------
Product_data:
['ProductKey', 'SKU', '

----
----

### 🧪 Initial Data Diagnostics

This section defines and runs a diagnostic function to scan all sheets in the Excel file for common data quality issues. It checks each column for:
- Missing values (`NaN`)
- Placeholder values (e.g., `-1`, `NA`, `[Not Applicable]`)

The result is a summary report showing where data cleaning is needed.

---

In [9]:
import pandas as pd

# Initial diagnostic: define a function to check for nulls and placeholders
def run_initial_diagnostics(file_path):
    placeholder_values = ['-1', '[Not Applicable]', 'NA']
    xls = pd.ExcelFile(file_path)
    report = []

    for sheet in xls.sheet_names:
        df = xls.parse(sheet)
        for col in df.columns:
            nulls = df[col].isnull().sum()
            placeholders = df[col].astype(str).isin(placeholder_values).sum()
            if nulls > 0 or placeholders > 0:
                report.append({
                    "Table": sheet,
                    "Column": col,
                    "Missing Values": nulls,
                    "Placeholder Values": placeholders
                })

    return pd.DataFrame(report)

# Run and display diagnostic report
input_path = r"data\AdventureWorks Sales.xlsx"
diagnostic_df = run_initial_diagnostics(input_path)
print("Initial Data Diagnostics:\n", diagnostic_df.to_string(index=False))

Initial Data Diagnostics:
         Table         Column  Missing Values  Placeholder Values
   Sales_data    ResellerKey               0               60398
   Sales_data    CustomerKey               0               60855
   Sales_data    ShipDateKey            2113                   0
Reseller_data    ResellerKey               0                   1
Reseller_data    Reseller ID               0                   1
Reseller_data  Business Type               0                   1
Reseller_data       Reseller               0                   1
Reseller_data           City               0                   1
Reseller_data State-Province               0                   1
Reseller_data Country-Region               0                   1
Reseller_data    Postal Code               0                   1
 Product_data          Color              56                   0
Customer_data    CustomerKey               0                   1
Customer_data    Customer ID               0                   

---
---

### ✅ Check for Duplicate Keys

This section checks whether the primary key columns `CustomerKey` and `ResellerKey` in their respective tables contain only unique values. Ensuring key uniqueness is critical for maintaining reliable joins and relationships between tables.

---

In [7]:
#Check for Duplicates or Inconsistent Data
customer_df["CustomerKey"].is_unique
reseller_df["ResellerKey"].is_unique

True

---
---

### 🧹 Data Cleaning and Transformation

This section performs the main data cleaning operations across multiple tables:

- **Customer_data**: Replaces placeholder `CustomerKey = -1` with a valid key and fills missing details with `'NA'`.
- **Reseller_data**: Replaces `ResellerKey = -1` similarly and sets related fields to `'NA'`.
- **Sales_data**:
  - Updates foreign keys (`CustomerKey`, `ResellerKey`) to match the cleaned dimension tables.
  - Converts date keys (`OrderDateKey`, `DueDateKey`, `ShipDateKey`) from `int` format (`YYYYMMDD`) to proper `date` objects.

Finally, it saves all the cleaned tables into a new Excel file called `AdventureWorks Sales_Cleaned.xlsx`.

---

In [8]:
import pandas as pd

# Define file paths
input_path = r"data\AdventureWorks Sales.xlsx"
output_path = r"data\AdventureWorks Sales_Cleaned.xlsx"

# Load Excel file
xls = pd.ExcelFile(input_path)
sheets = xls.sheet_names
dataframes = {sheet: xls.parse(sheet) for sheet in sheets}

# Clean Customer_data
customer_df = dataframes["Customer_data"]
customer_df.loc[customer_df["CustomerKey"] == -1, ["CustomerKey", "Customer ID", "Customer", "City", 
                                                   "State-Province", "Country-Region", "Postal Code"]] = [
    10999, "AW00010999", "NA", "NA", "NA", "NA", "NA"
]
# Convert Postal Code to string
customer_df["Postal Code"] = customer_df["Postal Code"].astype(str)
dataframes["Customer_data"] = customer_df

# Clean Reseller_data
reseller_df = dataframes["Reseller_data"]
reseller_df.loc[reseller_df["ResellerKey"] == -1, ["ResellerKey", "Reseller ID", "Business Type", "Reseller", 
                                                   "City", "State-Province", "Country-Region", "Postal Code"]] = [
    0, "AW00000000", "NA", "NA", "NA", "NA", "NA", "NA"
]
# Convert Postal Code to string
reseller_df["Postal Code"] = reseller_df["Postal Code"].astype(str)
dataframes["Reseller_data"] = reseller_df

# Update CustomerKey and ResellerKey in Sales_data
sales_df = dataframes["Sales_data"]
sales_df["CustomerKey"] = sales_df["CustomerKey"].replace(-1, 10999)
sales_df["ResellerKey"] = sales_df["ResellerKey"].replace(-1, 0)
dataframes["Sales_data"] = sales_df

# Convert integer keys to dates ──
sales_df["OrderDateKey"] = pd.to_datetime(sales_df["OrderDateKey"], format="%Y%m%d").dt.date
sales_df["DueDateKey"]   = pd.to_datetime(sales_df["DueDateKey"],   format="%Y%m%d").dt.date
sales_df["ShipDateKey"]  = pd.to_datetime(sales_df["ShipDateKey"],  format="%Y%m%d").dt.date

dataframes["Sales_data"] = sales_df

# Save cleaned data to new Excel file
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for sheet_name, df in dataframes.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("File is Cleaned and Saved in:", output_path)

File is Cleaned and Saved in: C:\Users\hp\Documents\Personal\Career\Portfolio\AdventureWorks\Data\AdventureWorks Sales_Cleaned.xlsx


---
---